In [39]:
import pandas as pd
import os

In [40]:
# number of partipants
'''

In the description there are
48 participants

'''

# Days recorded per participant
min_date = None
max_date = None

for file in os.listdir("./dataset/calendar/"):
    df = pd.read_csv(f"./dataset/calendar/{file}")
    if min_date == None and max_date == None:
        min_date = df['DATE'].min()
        max_date = df['DATE'].max()
    else:
        if df['DATE'].min() < min_date:
            min_date = df['DATE'].min()

        if df['DATE'].max() > max_date:
            max_date = df['DATE'].max()

print(f"Earliast Date: {min_date}")
print(f"Latest Date: {max_date}")
print()


# Demographic diversity (age, gender)

'''
48 undergrads and grad students
doesn't specify gender
'''

# Total datapoints and labels
 
total_datapoints = 0
total_labels = 0
for file in os.listdir("./dataset/calendar/"):
    df = pd.read_csv(f"./dataset/calendar/{file}")
    total_datapoints += df.shape[0]
    total_labels += len(df['ACCOUNT_LABEL'].unique())

print(f"Total Datapoints: {total_datapoints}")
print(f"Total Labels: {total_labels}")
print()

# Datapoints per label

per_label = pd.Series(dtype="float64")  

for file in os.listdir("./dataset/calendar/"):
    df = pd.read_csv(f"./dataset/calendar/{file}")

    counts = (
        df.groupby("ACCOUNT_LABEL", dropna=False)
          .agg(rows=("ACCOUNT_LABEL", 'size'))
    )

    per_label = per_label.add(counts['rows'], fill_value=0)

per_label = per_label.astype(int).sort_values(ascending=False)
print(per_label)

# Input streams (accelerometer, gyro, annotations, etc.) 

'''
activity
audio
conversation
bluetooth
gps
phonecharge
phonelock
wifi
wifi_location

'''

Earliast Date: 03/27/2013
Latest Date: 6/8/2013

Total Datapoints: 1687
Total Labels: 156

ACCOUNT_LABEL
1     563
2     413
3     247
4     178
5      92
6      70
7      39
8      30
9      20
10      8
12      5
11      5
13      3
14      3
15      3
16      2
17      2
18      2
19      1
20      1
dtype: int64


'\nactivity\naudio\nconversation\nbluetooth\ngps\nphonecharge\nphonelock\nwifi\nwifi_location\n\n'